In [2]:

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

imga='/content/sample_data/STA_0031.JPG'
imgb='/content/sample_data/STB_0032.JPG'
imgc='/content/sample_data/STC_0033.JPG'
imgd='/content/sample_data/STD_0034.JPG'
imge='/content/sample_data/STE_0035.JPG'
imgf='/content/sample_data/STF_0036.JPG'

img_pairs=[[imga,imgb],[imgb,imgc],[imgc,imgd],[imgd,imge],[imge,imgf]]

def plot_image(image_array):
  if type(image_array)==str:
    image_array=cv.imread(image_array)
  image_rgb = cv.cvtColor(image_array, cv.COLOR_BGR2RGB)
  plt.imshow(image_rgb)
  plt.axis('off')
  plt.show()

In [3]:

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

def calculate_homography(xy, x_y_):
        n = xy.shape[0]
        A = []
        for i in range(n):
            x, y = xy[i]
            u, v = x_y_[i]
            A.append([x, y, 1, 0, 0, 0, -u * x, -u * y, -u])
            A.append([0, 0, 0, x, y, 1, -v * x, -v * y, -v])
        A = np.array(A)
        U, S, V = np.linalg.svd(A)
        H = V[-1, :].reshape(3, 3)
        H = H / H[2, 2]
        return H

def ransac(img1_path,img2_path, threshold, num_iterations):
    img1 = cv.imread(img1_path, cv.IMREAD_UNCHANGED)
    img2 = cv.imread(img2_path, cv.IMREAD_UNCHANGED)

    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])

    xy = []
    x_y_ = []
    for i in good:
        indexa = i[0].queryIdx
        indexb = i[0].trainIdx
        xy.append(kp1[indexa].pt)
        x_y_.append(kp2[indexb].pt)

    xy = np.array(xy)
    x_y_ = np.array(x_y_)
    best_H = None
    max_inliers = 0
    for _ in range(num_iterations):
        sample_indices = np.random.choice(xy.shape[0], 4, replace=False)
        sample_xy = xy[sample_indices]
        sample_x_y_ = x_y_[sample_indices]
        H = calculate_homography(sample_xy, sample_x_y_)
        inliers = 0
        for i in range(xy.shape[0]):
            x, y = xy[i]
            u, v = x_y_[i]
            p = np.array([x, y, 1])
            transformed_point = np.dot(H, p)
            transformed_point = transformed_point / transformed_point[2]
            error = np.linalg.norm(transformed_point[:2] - np.array([u, v]))
            if error < threshold:
                inliers += 1
        if inliers > max_inliers:
            max_inliers = inliers
            best_H = H
    return best_H, max_inliers

def stitch_images(img1_path, img2_path, self):
    img1 = cv.imread(img1_path, cv.IMREAD_UNCHANGED)
    img2 = cv.imread(img2_path, cv.IMREAD_UNCHANGED)

    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])

    xy = []
    x_y_ = []
    for i in good:
        indexa = i[0].queryIdx
        indexb = i[0].trainIdx
        xy.append(kp1[indexa].pt)
        x_y_.append(kp2[indexb].pt)

    xy = np.array(xy)
    x_y_ = np.array(x_y_)

    # def calculate_homography(xy, x_y_):
    #     n = xy.shape[0]
    #     A = []
    #     for i in range(n):
    #         x, y = xy[i]
    #         u, v = x_y_[i]
    #         A.append([x, y, 1, 0, 0, 0, -u * x, -u * y, -u])
    #         A.append([0, 0, 0, x, y, 1, -v * x, -v * y, -v])
    #     A = np.array(A)
    #     U, S, V = np.linalg.svd(A)
    #     H = V[-1, :].reshape(3, 3)
    #     H = H / H[2, 2]
    #     return H

    # def ransac(xy, x_y_, threshold, num_iterations):
    #     best_H = None
    #     max_inliers = 0
    #     for _ in range(num_iterations):
    #         sample_indices = np.random.choice(xy.shape[0], 4, replace=False)
    #         sample_xy = xy[sample_indices]
    #         sample_x_y_ = x_y_[sample_indices]
    #         H = calculate_homography(sample_xy, sample_x_y_)
    #         inliers = 0
    #         for i in range(xy.shape[0]):
    #             x, y = xy[i]
    #             u, v = x_y_[i]
    #             p = np.array([x, y, 1])
    #             transformed_point = np.dot(H, p)
    #             transformed_point = transformed_point / transformed_point[2]
    #             error = np.linalg.norm(transformed_point[:2] - np.array([u, v]))
    #             if error < threshold:
    #                 inliers += 1
    #         if inliers > max_inliers:
    #             max_inliers = inliers
    #             best_H = H
    #     return best_H, max_inliers

    #H_manual_ransac = ransac(xy, x_y_, threshold=5.0, num_iterations=1000)
    H_manual_ransac=ransacs[self]
    rows, cols = img1.shape[:2]
    coords = np.array([[0, 0, 1], [cols, 0, 1], [0, rows, 1], [cols, rows, 1]])
    warped_coords = np.dot(H_manual_ransac, coords.T).T
    warped_coords /= warped_coords[:, 2:3]
    all_coords = np.concatenate((coords, warped_coords))
    min_x = np.min(all_coords[:, 0])
    max_x = np.max(all_coords[:, 0])
    min_y = np.min(all_coords[:, 1])
    max_y = np.max(all_coords[:, 1])
    warped_canvas_size = (int(max_x - min_x), int(max_y - min_y))
    translation = np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]])
    print(translation)
    warped_img = cv.warpPerspective(img1, translation.dot(H_manual_ransac), warped_canvas_size)
    shifted_ref_img = cv.warpPerspective(img2, translation, warped_canvas_size)

    def single_weights_array(size):
        if size % 2 == 1:
            return np.concatenate([np.linspace(0, 1, (size + 1) // 2), np.linspace(1, 0, (size + 1) // 2)[1:]])
        else:
            return np.concatenate([np.linspace(0, 1, size // 2), np.linspace(1, 0, size // 2)])

    def single_weights_matrix(shape):
        return np.array(single_weights_array(shape[0])[:, np.newaxis] @ single_weights_array(shape[1])[:, np.newaxis].T)

    mask = single_weights_matrix(shifted_ref_img.shape[:2])
    mask1 = cv.warpPerspective(mask, translation, warped_canvas_size)
    mask2 = cv.warpPerspective(mask, H_manual_ransac, warped_canvas_size)
    mask1 = mask1 / mask1.max()
    mask2 = mask2 / mask2.max()
    mask3 = np.divide(mask1, (mask1 + mask2), where=(mask1) != 0)
    mask3 = np.array([mask3, mask3, mask3]).transpose(1, 2, 0)
    mask3 = mask3 / mask3.max()
    masked_ref = mask3 * shifted_ref_img
    mask4 = np.divide(mask2, (mask1 + mask2), where=(mask2) != 0)
    mask4 = np.array([mask4, mask4, mask4]).transpose(1, 2, 0)
    mask4 = mask4 / mask4.max()
    masked_warped = (mask4) * warped_img
    added_image = masked_ref + masked_warped

    return  added_image.astype(np.uint8)

In [4]:
ransacc=[]

for i in img_pairs:
  current=np.array(ransac(i[0],i[1],5.0,1000)[0])
  ransacc.append(current)
  print(current.shape)


ransacs=[ransacc[0]]



error: OpenCV(4.10.0) /io/opencv/modules/features2d/src/sift.dispatch.cpp:512: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'detectAndCompute'


In [5]:
for i in range(len(ransacc)-1):
  ransacs.append(np.array(ransacc[i])@np.array(ransacc[i+1]))

In [6]:
result_images=[imga]
images=[imgb,imgc,imgd,imge,imgf]

for i in range (0,2):
  prev=result_images[-1]
  path='/content/sample_data/'+str(i+1)+'.JPG'
  current=images[i]
  plot_image(current)
  added=stitch_images(prev,current,i)
  cv.imwrite(path,added)
  result_images.append(path)
  plot_image(added)


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
current=stitch_images(imga,imgb)
cv.imwrite('/content/sample_data/resultab.JPG', current)
alphabet=[imgc,imgd,imge,imgf]
plot_image(current)
path='/content/sample_data/resultab.JPG'
strings=['abc','abcd','abcde','abcdef']

for i in range(0,4):
  current=stitch_images(path,alphabet[i])
  print(strings[i])
  plot_image(current)
  path='/content/sample_data/result'+strings[i]+'.jpg'
  cv.imwrite(path,current)